In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
#X = tf.placeholder('float')
#X= tf.reshape(X, shape=[-1, 64, 64, 20, 1])
X = tf.placeholder(shape=[None,64,64,20, 1], dtype=tf.float32, name="X")
print(X.shape)

In [3]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 24 * 24*2  # 1152 primary capsules
caps1_n_dims = 8

In [4]:
conv1_params = {
    "filters": 256,
    "kernel_size": [9,9,9],
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": [9,9,9],
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}

In [5]:
conv1 = tf.layers.conv3d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv3d(conv1, name="conv2", **conv2_params)

In [6]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")

In [7]:
print(conv1.shape)
print(conv2.shape)
print(caps1_raw.shape)

(?, 56, 56, 12, 256)
(?, 24, 24, 2, 256)
(?, 36864, 8)


In [8]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        result_vector=squash_factor * unit_vector
        #print(result_vector)
        return result_vector

In [9]:
caps1_output = squash(caps1_raw, name="caps1_output")

In [10]:
print(caps1_output.shape)

(?, 36864, 8)


In [11]:
caps2_n_caps = 2
caps2_n_dims = 16

In [12]:
init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

In [13]:
print(W_init.shape)
print(W.shape)

(1, 36864, 2, 16, 8)
(1, 36864, 2, 16, 8)


In [14]:
batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

In [15]:
print(batch_size.shape)
print(W_tiled.shape)

()
(?, 36864, 2, 16, 8)


In [16]:
caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

In [17]:
print(caps1_output_expanded.shape)
print(caps1_output_tile.shape)
print(caps1_output_tiled.shape)

(?, 36864, 8, 1)
(?, 36864, 1, 8, 1)
(?, 36864, 2, 8, 1)


In [18]:
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")

In [19]:
print(caps2_predicted.shape)

(?, 36864, 2, 16, 1)


In [20]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")

In [21]:
raw_weights

<tf.Tensor 'raw_weights:0' shape=(?, 36864, 2, 1, 1) dtype=float32>

In [22]:
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")

In [23]:
routing_weights

<tf.Tensor 'routing_weights:0' shape=(?, 36864, 2, 1, 1) dtype=float32>

In [24]:
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")

In [25]:
print(weighted_predictions)
print(weighted_sum)

Tensor("weighted_predictions:0", shape=(?, 36864, 2, 16, 1), dtype=float32)
Tensor("weighted_sum:0", shape=(?, 1, 2, 16, 1), dtype=float32)


In [26]:
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")

In [27]:
caps2_output_round_1

<tf.Tensor 'caps2_output_round_1/mul:0' shape=(?, 1, 2, 16, 1) dtype=float32>

In [28]:
caps2_predicted

<tf.Tensor 'caps2_predicted:0' shape=(?, 36864, 2, 16, 1) dtype=float32>

In [29]:
caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

In [30]:
caps2_output_round_1_tiled

<tf.Tensor 'caps2_output_round_1_tiled:0' shape=(?, 36864, 2, 16, 1) dtype=float32>

In [31]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

In [32]:
agreement

<tf.Tensor 'agreement:0' shape=(?, 36864, 2, 1, 1) dtype=float32>

In [33]:
raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

In [34]:
raw_weights_round_2

<tf.Tensor 'raw_weights_round_2:0' shape=(?, 36864, 2, 1, 1) dtype=float32>

In [35]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")

In [36]:
print(routing_weights_round_2)
print(weighted_predictions_round_2)
print(weighted_sum_round_2)
print(caps2_output_round_2)

Tensor("routing_weights_round_2:0", shape=(?, 36864, 2, 1, 1), dtype=float32)
Tensor("weighted_predictions_round_2:0", shape=(?, 36864, 2, 16, 1), dtype=float32)
Tensor("weighted_sum_round_2:0", shape=(?, 1, 2, 16, 1), dtype=float32)
Tensor("caps2_output_round_2/mul:0", shape=(?, 1, 2, 16, 1), dtype=float32)


In [37]:
caps2_output = caps2_output_round_2

In [38]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

In [39]:
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")

In [40]:
y_proba


<tf.Tensor 'y_proba/Sqrt:0' shape=(?, 1, 2, 1) dtype=float32>

In [41]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")

In [42]:
y_proba_argmax

<tf.Tensor 'y_proba_1:0' shape=(?, 1, 1) dtype=int64>

In [43]:
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")

In [44]:
y_pred

<tf.Tensor 'y_pred:0' shape=(?,) dtype=int64>

In [45]:
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

In [46]:
y

<tf.Tensor 'y:0' shape=(?,) dtype=int64>

In [47]:
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

In [48]:
T = tf.one_hot(y, depth=caps2_n_caps, name="T")

In [49]:
T

<tf.Tensor 'T:0' shape=(?, 2) dtype=float32>

In [50]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

In [51]:
caps2_output_norm

<tf.Tensor 'caps2_output_norm/Sqrt:0' shape=(?, 1, 2, 1, 1) dtype=float32>

In [52]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

In [53]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 2),
                           name="present_error")
present_error

<tf.Tensor 'present_error:0' shape=(?, 2) dtype=float32>

In [54]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 2),
                          name="absent_error")

In [55]:
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

In [56]:
L

<tf.Tensor 'L:0' shape=(?, 2) dtype=float32>

In [57]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")
margin_loss

<tf.Tensor 'margin_loss:0' shape=() dtype=float32>

In [58]:
mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")
mask_with_labels

<tf.Tensor 'mask_with_labels:0' shape=() dtype=bool>

In [59]:
reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")
reconstruction_targets

<tf.Tensor 'reconstruction_targets/Merge:0' shape=(?,) dtype=int64>

In [60]:
reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

In [61]:
reconstruction_mask

<tf.Tensor 'reconstruction_mask:0' shape=(?, 2) dtype=float32>

In [62]:
#compare
caps2_output

<tf.Tensor 'caps2_output_round_2/mul:0' shape=(?, 1, 2, 16, 1) dtype=float32>

In [63]:
reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")

In [64]:
reconstruction_mask_reshaped

<tf.Tensor 'reconstruction_mask_reshaped:0' shape=(?, 1, 2, 1, 1) dtype=float32>

In [65]:
caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

In [66]:
caps2_output_masked

<tf.Tensor 'caps2_output_masked:0' shape=(?, 1, 2, 16, 1) dtype=float32>

In [67]:
decoder_input = tf.reshape(caps2_output_masked,
                           [-1, caps2_n_caps * caps2_n_dims],
                           name="decoder_input")

In [68]:
decoder_input

<tf.Tensor 'decoder_input:0' shape=(?, 32) dtype=float32>

In [69]:
n_hidden1 = 512
n_hidden2 = 1024
n_output = 64 * 64 * 20

In [70]:
with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                              activation=tf.nn.relu,
                              name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu,
                              name="hidden2")
    decoder_output = tf.layers.dense(hidden2, n_output,
                                     activation=tf.nn.sigmoid,
                                     name="decoder_output")

In [71]:
print(hidden1)
print(hidden2)
print(decoder_output)

Tensor("decoder/hidden1/Relu:0", shape=(?, 512), dtype=float32)
Tensor("decoder/hidden2/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("decoder/decoder_output/Sigmoid:0", shape=(?, 81920), dtype=float32)


In [72]:
X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                    name="reconstruction_loss")

In [73]:
print(X_flat)
print(squared_difference)
print(reconstruction_loss)

Tensor("X_flat:0", shape=(?, 81920), dtype=float32)
Tensor("squared_difference:0", shape=(?, 81920), dtype=float32)
Tensor("reconstruction_loss:0", shape=(), dtype=float32)


In [74]:
alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")

In [75]:
loss

<tf.Tensor 'loss:0' shape=() dtype=float32>

In [76]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64), name="accuracy")

In [77]:
print(correct)
print(accuracy)

Tensor("correct:0", shape=(?,), dtype=bool)
Tensor("accuracy:0", shape=(), dtype=float64)


In [78]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

In [79]:
init = tf.global_variables_initializer()

In [80]:
n_epochs = 10
batch_size = 50

In [81]:
imageData = np.load('imageDataNew60-64-64-20.npy')

In [82]:
n_epochs = 10
batch_size = 10
trainingData = imageData[:-100]
validationData = imageData[-100:]
print(len(trainingData))
print(len(validationData))

1297
100


In [83]:
n_iterations_per_epoch =  len(trainingData)// batch_size
n_iterations_validation = len(validationData)// batch_size
print(n_iterations_per_epoch)
print(n_iterations_validation)

129
10


In [84]:
best_loss_val = np.infty
#checkpoint_path = "./my_capsule_network"

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for iteration,data in enumerate(trainingData):
            X_batch = data[0]
            y_batch = data[1]
            # Run the training operation and measure the loss:
            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 64, 64,20, 1]),
                           y: y_batch,
                           mask_with_labels: True})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(iteration, n_iterations_per_epoch,
                    iteration * 100 / n_iterations_per_epoch,loss_train),
                  end="")

        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        for iteration,valdata in enumerate(validationData):
            X_batch = valdata[0]
            y_batch = valdata[1]
            loss_val, acc_val = sess.run(
                    [loss, accuracy],
                    feed_dict={X: X_batch.reshape([-1, 64, 64,20, 1]),
                               y: y_batch})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                  end=" " * 10)
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

Epoch: 1  Val accuracy: 50.0000%  Loss: 461.100739 (improved)
Iteration: 39/129 (30.2%)  Loss: 263.54846

KeyboardInterrupt: 

In [ ]:
for data in trainingData[:2]:
    X = data[0]
    Y = data[1]

In [ ]:
Y